In [19]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import math
import folium
import webbrowser
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageTk
import threading
import matplotlib.pyplot as plt
from flask import Flask, jsonify
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

# Constants and tunable parameters
K_MEANS_CLUSTERS = 6
DILATION_KERNEL_SIZE = (15, 15)
DILATION_ITERATIONS = 1
MIN_LINE_LENGTH = 1
MAX_LINE_GAP = 2
SLOPE_THRESHOLD = 0.1
# Parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

class CropRowDetector:
    def __init__(self):
        self.image_path = None
        self.cam_x = 0.0
        self.cam_y = 0.0
        self.cam_h = 0.0
        self.resolution = 0.0
        self.cam_angle = 0.0
        self.global_points = []
        # Parameters for the model input
        self.IMG_WIDTH = 128
        self.IMG_HEIGHT = 128
        self.IMG_CHANNELS = 3
        # Load the trained model for binary mask prediction
        model_path = 'C:/Users/lenovo/Documents/GitHub/Mechatronics/Codes/crop_row_detection_model.h5'
        self.model = tf.keras.models.load_model(model_path, compile=False)
        
    def predict_binary_mask(self, img):
        """
        Preprocess the image for the model, predict the binary mask, and return it.
        """
        # Resize the image for the model input (128x128x3)
        img_resized = resize(img, (self.IMG_HEIGHT, self.IMG_WIDTH), mode='constant', preserve_range=True)
        img_resized = img_resized[:, :, :self.IMG_CHANNELS]  # Ensure it has 3 channels

        # Add batch dimension (model expects batch, height, width, channels)
        img_resized = np.expand_dims(img_resized, axis=0)

        # Get model prediction
        prediction = self.model.predict(img_resized)[0]

        # Convert the prediction to a binary mask
        binary_mask = (prediction > 0.5).astype(np.uint8)

        return binary_mask  # Keep the mask in the size (128x128)

    
    def set_global_points(self, points):
        self.global_points = points

    def find_crop_rows(self, binary_mask):
        crop_rows = cv2.HoughLinesP(binary_mask, rho=1, theta=np.pi/180, threshold=50, minLineLength=MIN_LINE_LENGTH, maxLineGap=MAX_LINE_GAP)
        return crop_rows if crop_rows is not None else []

    def merge_lines(self, lines, eps=0.5, min_samples=2, slope_threshold=SLOPE_THRESHOLD):
        slopes = []
        intercepts = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 != x1:
                    slope = (y2 - y1) / (x2 - x1)
                    intercept = y1 - slope * x1
                    slopes.append(slope)
                    intercepts.append(intercept)
        avg_slope = np.mean(slopes)
        filtered_lines = [(slope, intercept) for slope, intercept in zip(slopes, intercepts) if abs(slope - avg_slope) <= slope_threshold]
        features = np.array(filtered_lines)
        if len(features) == 0:
            return []
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(features_scaled)
        labels = db.labels_
        unique_labels = set(labels)
        merged_lines = []
        for label in unique_labels:
            if label == -1:
                continue
            cluster_lines = features[labels == label]
            avg_slope = np.mean(cluster_lines[:, 0])
            avg_intercept = np.mean(cluster_lines[:, 1])
            merged_lines.append((avg_slope, avg_intercept))
        return merged_lines

    def draw_lines(self, image, lines, color):
        for slope, intercept in lines:
            x1 = 0
            y1 = int(intercept)
            x2 = image.shape[1]
            y2 = int(slope * x2 + intercept)
            cv2.line(image, (x1, y1), (x2, y2), color, 5)

    def draw_middle_lines(self, image, merged_lines, middle_line_color=(255, 0, 0)):
        merged_lines.sort(key=lambda x: x[1])
        middle_lines = []
        for i in range(len(merged_lines) - 1):
            slope1, intercept1 = merged_lines[i]
            slope2, intercept2 = merged_lines[i + 1]
            avg_slope = (slope1 + slope2) / 2
            avg_intercept = (intercept1 + intercept2) / 2
            middle_lines.append((avg_slope, avg_intercept))
        self.draw_lines(image, middle_lines, middle_line_color)
        return middle_lines

    def draw_points_on_lines(self, image, lines, num_points=10, point_color=(0, 0, 255)):
        height, width, _ = image.shape
        points = []
        for slope, intercept in lines:
            x_values = np.linspace(0, width - 1, num_points)
            y_values = slope * x_values + intercept
            line_points = []
            for x, y in zip(x_values, y_values):
                x = int(x)
                y = int(y)
                if 0 <= x < width and 0 <= y < height:
                    cv2.circle(image, (x, y), 10, point_color, -1)
                    line_points.append((x, y))
            points.extend(line_points)
        return points

    def pixel_to_global(self, y_pixel, x_pixel, cam_x, cam_y, cam_h, resolution, cam_angle):
        meter_to_global = 0.00001
        pixel_size_meters = math.tan(cam_angle) * cam_h / (resolution / 2)
        y_centered = resolution / 2 - y_pixel
        x_centered = x_pixel - resolution / 2
        y_meters = y_centered * pixel_size_meters
        x_meters = x_centered * pixel_size_meters
        pixel_global_x = cam_x + y_meters * meter_to_global
        pixel_global_y = cam_y + x_meters * meter_to_global
        return pixel_global_x, pixel_global_y

    def process_image(self, image_path, cam_x, cam_y, cam_h, resolution, cam_angle):
        """
        Process the image, predict the binary mask, and find crop rows.
        """
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Unable to read image at {image_path}")
            return None, None, None, None, None

        # Use the model to predict the binary mask
        binary_mask = self.predict_binary_mask(img)  # Binary mask size (128x128)

        # Continue processing (dilation, Hough lines, etc.)
        kernel = np.ones(DILATION_KERNEL_SIZE, np.uint8)
        binary_mask_dilated = cv2.dilate(binary_mask, kernel, iterations=DILATION_ITERATIONS)

        crop_rows = self.find_crop_rows(binary_mask_dilated)  # Detect crop rows from binary mask
        merged_lines = self.merge_lines(crop_rows, eps=0.5, min_samples=2)
        middle_lines = self.draw_middle_lines(img, merged_lines) if merged_lines else []
        points = self.draw_points_on_lines(img, middle_lines) if middle_lines else []

        # Convert pixel points to global coordinates
        global_points = [self.pixel_to_global(y, x, cam_x, cam_y, cam_h, resolution, cam_angle) for x, y in points]

        # Save processed images
        cv2.imwrite("processed_image_original.png", img)
        cv2.imwrite("processed_image_binary_mask.png", binary_mask)

        # If crop rows are detected, save the image with crop row lines
        if merged_lines:
            img_with_rows = img.copy()
            self.draw_lines(img_with_rows, merged_lines, (0, 0, 255))
            cv2.imwrite("processed_image_with_rows.png", img_with_rows)

            if middle_lines:
                img_with_middle_lines = img.copy()
                self.draw_lines(img_with_middle_lines, middle_lines, (255, 0, 0))
                cv2.imwrite("processed_image_with_middle_lines.png", img_with_middle_lines)

        return img, binary_mask, None, merged_lines, global_points



class CropRowDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Crop Row Detection App")
        self.create_widgets()
        self.detector = CropRowDetector()
        self.cap = None
        self.frame = None
        self.stop_event = threading.Event()
        self.output_dir = "C:/Users/lenovo/Documents/Mechatronics/Source_Directory"
        self.flask_thread = threading.Thread(target=self.run_flask_server, daemon=True)
        self.flask_thread.start()
        
    def run_flask_server(self):
        app = Flask(__name__)
    
        @app.route('/data', methods=['GET'])
        def get_data():
            # Return the latest global points data
            return jsonify({"points": self.detector.global_points})
    
        app.run(host='0.0.0.0', port=5000)
        
    def create_widgets(self):
        self.label = tk.Label(self.root, text="Crop Row Detection App", font=("Helvetica", 16))
        self.label.pack(pady=10)
    
        self.choose_image_button = tk.Button(self.root, text="Choose Image", command=self.choose_image)
        self.choose_image_button.pack(pady=5)
    
        self.start_live_button = tk.Button(self.root, text="Start Live Stream", command=self.start_live_stream)
        self.start_live_button.pack(pady=5)
    
        self.capture_button = tk.Button(self.root, text="Capture Photo", command=self.capture_photo)
        self.capture_button.pack(pady=5)
    
        self.param_frame = tk.Frame(self.root)
        self.param_frame.pack(pady=10)
    
        tk.Label(self.param_frame, text="Camera X:").grid(row=0, column=0, padx=5, pady=5)
        self.cam_x_entry = tk.Entry(self.param_frame)
        self.cam_x_entry.grid(row=0, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Y:").grid(row=1, column=0, padx=5, pady=5)
        self.cam_y_entry = tk.Entry(self.param_frame)
        self.cam_y_entry.grid(row=1, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Height:").grid(row=2, column=0, padx=5, pady=5)
        self.cam_h_entry = tk.Entry(self.param_frame)
        self.cam_h_entry.grid(row=2, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Resolution:").grid(row=3, column=0, padx=5, pady=5)
        self.resolution_entry = tk.Entry(self.param_frame)
        self.resolution_entry.grid(row=3, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Angle:").grid(row=4, column=0, padx=5, pady=5)
        self.cam_angle_entry = tk.Entry(self.param_frame)
        self.cam_angle_entry.grid(row=4, column=1, padx=5, pady=5)
    
        self.process_image_button = tk.Button(self.root, text="Process Image", command=self.process_image)
        self.process_image_button.pack(pady=5)
    
        self.image_label = tk.Label(self.root)
        self.image_label.pack(pady=10)
        
    def choose_image(self):
        self.detector.image_path = filedialog.askopenfilename(initialdir=self.output_dir, title="Select an Image", filetypes=[("Image files", "*.jpg; *.jpeg; *.png")])
        self.display_image(self.detector.image_path)

    def start_live_stream(self):
        # Find the OBS Virtual Camera index
        self.camera_index = self.find_virtual_camera()
        if self.camera_index == -1:
            messagebox.showerror("Error", "OBS Virtual Camera not found.")
            return
        
        # Open the camera
        self.cap = cv2.VideoCapture(self.camera_index)
        if not self.cap.isOpened():
            messagebox.showerror("Error", f"Unable to open camera at index {self.camera_index}.")
            return
        
        self.stop_event.clear()
        self.show_live_stream()

    def show_live_stream(self):
        if self.stop_event.is_set():
            return
        ret, frame = self.cap.read()
        if ret:
            self.frame = frame
            cv2.imshow("Live Stream", frame)
        self.root.after(10, self.show_live_stream)

    def find_virtual_camera(self, start_index=1, max_index=10):
        """Try to find the OBS Virtual Camera by iterating over possible device indices."""
        for index in range(start_index, max_index):
            cap = cv2.VideoCapture(index)
            if cap.isOpened():
                print(f"Found camera at index {index}")
                
                # Check if this is the OBS Virtual Camera by capturing a frame and analyzing its content
                ret, frame = cap.read()
                if ret:
                    # Check if frame is not empty or perform other checks
                    if cv2.countNonZero(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) > 0:
                        print(f"Camera at index {index} seems to be working.")
                        cap.release()
                        return index
                    else:
                        print(f"Camera at index {index} is not the OBS Virtual Camera.")
                else:
                    print(f"Unable to read frame from camera at index {index}.")
                
                cap.release()
    
        print("No OBS Virtual Camera found.")
        return -1

    def capture_photo(self):
        if self.frame is not None:
            image_path = os.path.join(self.output_dir, "captured_image.png")
            cv2.imwrite(image_path, self.frame)
            self.detector.image_path = image_path
            self.display_image(image_path)
            messagebox.showinfo("Image Captured", f"Image saved to {image_path}")

    def display_image(self, image_path):
        img = Image.open(image_path)
        img.thumbnail((400, 400))
        img = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img)
        self.image_label.image = img

    def process_image(self):
        if self.detector.image_path is None:
            messagebox.showerror("Error", "Please select or capture an image first.")
            return
    
        # Retrieve parameters from entry fields
        try:
            cam_x = float(self.cam_x_entry.get())
            cam_y = float(self.cam_y_entry.get())
            cam_h = float(self.cam_h_entry.get())
            resolution = float(self.resolution_entry.get())
            cam_angle = float(self.cam_angle_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical values for all parameters.")
            return
    
        # Call function to process image
        img, preprocessed_image, segmented_image, merged_lines, global_points = self.detector.process_image(
            self.detector.image_path, cam_x, cam_y, cam_h, resolution, cam_angle
        )
    
        # Update global points
        self.detector.global_points = global_points
    
        if global_points:
            # Save map and plots
            map_filename = os.path.abspath("crop_row_map.html")
            self.create_map(map_filename, global_points)
            
            plot_filename = self.save_plots(self.detector.image_path)
            self.generate_html_report(plot_filename, map_filename)
        else:
            messagebox.showwarning("Warning", "No global points detected.")
        
    def create_map(self, map_filename, points):
        m = folium.Map(location=[points[0][0], points[0][1]], zoom_start=18)
    
        for point in points:
            folium.Marker(location=[point[0], point[1]]).add_to(m)
    
        m.save(map_filename)
    
    def on_closing(self):
        if self.cap is not None:
            self.stop_event.set()
            self.cap.release()
            cv2.destroyAllWindows()
        self.root.destroy()

    
    def save_plots(self, image_path):
        image, preprocessed_image, segmented_image, merged_lines, _ = self.detector.process_image(
            image_path,
            float(self.cam_x_entry.get()),
            float(self.cam_y_entry.get()),
            float(self.cam_h_entry.get()),
            float(self.resolution_entry.get()),
            float(self.cam_angle_entry.get())
        )
        
        if image is not None:
            # Save the plots
            fig, axes = plt.subplots(1, 5, figsize=(25, 8))
            axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[0].set_title('Original Image')
            axes[0].axis('off')

            axes[1].imshow(preprocessed_image, cmap='gray')
            axes[1].set_title('Preprocessed Image')
            axes[1].axis('off')

            axes[2].imshow(segmented_image, cmap='gray')
            axes[2].set_title('Segmented Image (K-Means)')
            axes[2].axis('off')

            if merged_lines:
                for slope, intercept in merged_lines:
                    x1 = 0
                    y1 = int(intercept)
                    x2 = image.shape[1]
                    y2 = int(slope * x2 + intercept)
                    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 10)

            axes[3].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[3].set_title('Detected Crop Rows')
            axes[3].axis('off')

            middle_lines = self.detector.draw_middle_lines(image, merged_lines) if merged_lines else []
            if middle_lines:
                for slope, intercept in middle_lines:
                    x1 = 0
                    y1 = int(intercept)
                    x2 = image.shape[1]
                    y2 = int(slope * x2 + intercept)
                    cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

            axes[4].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[4].set_title('Middle Lines')
            axes[4].axis('off')

            plt.tight_layout()
            plot_filename = os.path.join(self.output_dir, "plots.png")
            plt.savefig(plot_filename)
            plt.close()
            
            return plot_filename

    def generate_html_report(self, plot_filename, map_filename):
        html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Crop Row Detection Report</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 0;
                    background-color: #f4f4f4;
                }}
                .container {{
                    width: 80%;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: #fff;
                    border-radius: 8px;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                }}
                h1 {{
                    color: #4CAF50;
                    text-align: center;
                }}
                h2 {{
                    color: #333;
                }}
                .plot-container, .map-container {{
                    width: 100%;
                    max-width: 800px;
                    margin: 20px auto;
                    text-align: center;
                }}
                .plot-container img {{
                    max-width: 100%;
                    height: auto;
                    border-radius: 8px;
                }}
                .map-container iframe {{
                    width: 100%;
                    height: 500px;
                    border: none;
                    border-radius: 8px;
                }}
                .footer {{
                    text-align: center;
                    padding: 10px;
                    color: #666;
                }}
                .footer p {{
                    margin: 0;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Crop Row Detection Report</h1>
                <h2>Map Visualization</h2>
                <div class="map-container">
                    <iframe src="{map_filename}" title="Crop Row Map"></iframe>
                </div>
                <h2>Plots</h2>
                <div class="plot-container">
                    <img src="{plot_filename}" alt="Plots">
                </div>
                <div class="footer">
                    <p>Generated with love for agriculture.</p>
                </div>
            </div>
        </body>
        </html>
        """
        
        report_filename = os.path.join(self.output_dir, "report.html")
        with open(report_filename, "w") as file:
            file.write(html_content)
        
        webbrowser.open(report_filename)
        
            
    def process_image(self):
        if self.detector.image_path is None:
            messagebox.showerror("Error", "Please select or capture an image first.")
            return
    
        # Retrieve parameters from entry fields
        try:
            cam_x = float(self.cam_x_entry.get())
            cam_y = float(self.cam_y_entry.get())
            cam_h = float(self.cam_h_entry.get())
            resolution = float(self.resolution_entry.get())
            cam_angle = float(self.cam_angle_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical values for all parameters.")
            return
    
        # Call function to process image
        img, preprocessed_image, segmented_image, merged_lines, global_points = self.detector.process_image(
            self.detector.image_path, cam_x, cam_y, cam_h, resolution, cam_angle
        )
    
        if global_points:
            # Save map and plots
            map_filename = os.path.abspath("crop_row_map.html")
            self.create_map(map_filename, global_points)
            
            plot_filename = self.save_plots(self.detector.image_path)
            self.generate_html_report(plot_filename, map_filename)
        else:
            messagebox.showwarning("Warning", "No global points detected.")

                    
if __name__ == "__main__":
    root = tk.Tk()
    app = CropRowDetectionApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.9:5000
Press CTRL+C to quit


1/1 [==============================] - 0s 188ms/step


In [18]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import math
import folium
import webbrowser
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageTk
import threading
import matplotlib.pyplot as plt
from flask import Flask, jsonify
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize


# Constants and tunable parameters
K_MEANS_CLUSTERS = 6
DILATION_KERNEL_SIZE = (15, 15)
DILATION_ITERATIONS = 1
MIN_LINE_LENGTH = 1
MAX_LINE_GAP = 2
SLOPE_THRESHOLD = 0.1

class CropRowDetector:
    def __init__(self):
        self.image_path = None
        self.cam_x = 0.0
        self.cam_y = 0.0
        self.cam_h = 0.0
        self.resolution = 0.0
        self.cam_angle = 0.0
        self.global_points = []
        
    def set_global_points(self, points):
        self.global_points = points
        
    def preprocess_image(self, img):
        hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])
        green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
        kernel = np.ones((5, 5), np.uint8)
        green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel)
        green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
        return green_mask

    def apply_kmeans(self, image):
        pixel_values = image.reshape((-1, 1))
        pixel_values = np.float32(pixel_values)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        _, labels, centers = cv2.kmeans(pixel_values, K_MEANS_CLUSTERS, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        centers = np.uint8(centers)
        segmented_image = centers[labels.flatten()]
        segmented_image = segmented_image.reshape(image.shape)
        _, binary_mask = cv2.threshold(segmented_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return binary_mask

    def find_crop_rows(self, binary_mask):
        crop_rows = cv2.HoughLinesP(binary_mask, rho=1, theta=np.pi/180, threshold=50, minLineLength=MIN_LINE_LENGTH, maxLineGap=MAX_LINE_GAP)
        return crop_rows if crop_rows is not None else []

    def merge_lines(self, lines, eps=0.5, min_samples=2, slope_threshold=SLOPE_THRESHOLD):
        slopes = []
        intercepts = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 != x1:
                    slope = (y2 - y1) / (x2 - x1)
                    intercept = y1 - slope * x1
                    slopes.append(slope)
                    intercepts.append(intercept)
        avg_slope = np.mean(slopes)
        filtered_lines = [(slope, intercept) for slope, intercept in zip(slopes, intercepts) if abs(slope - avg_slope) <= slope_threshold]
        features = np.array(filtered_lines)
        if len(features) == 0:
            return []
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(features)
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(features_scaled)
        labels = db.labels_
        unique_labels = set(labels)
        merged_lines = []
        for label in unique_labels:
            if label == -1:
                continue
            cluster_lines = features[labels == label]
            avg_slope = np.mean(cluster_lines[:, 0])
            avg_intercept = np.mean(cluster_lines[:, 1])
            merged_lines.append((avg_slope, avg_intercept))
        return merged_lines

    def draw_lines(self, image, lines, color):
        for slope, intercept in lines:
            x1 = 0
            y1 = int(intercept)
            x2 = image.shape[1]
            y2 = int(slope * x2 + intercept)
            cv2.line(image, (x1, y1), (x2, y2), color, 5)

    def draw_middle_lines(self, image, merged_lines, middle_line_color=(255, 0, 0)):
        merged_lines.sort(key=lambda x: x[1])
        middle_lines = []
        for i in range(len(merged_lines) - 1):
            slope1, intercept1 = merged_lines[i]
            slope2, intercept2 = merged_lines[i + 1]
            avg_slope = (slope1 + slope2) / 2
            avg_intercept = (intercept1 + intercept2) / 2
            middle_lines.append((avg_slope, avg_intercept))
        self.draw_lines(image, middle_lines, middle_line_color)
        return middle_lines

    def draw_points_on_lines(self, image, lines, num_points=10, point_color=(0, 0, 255)):
        height, width, _ = image.shape
        points = []
        for slope, intercept in lines:
            x_values = np.linspace(0, width - 1, num_points)
            y_values = slope * x_values + intercept
            line_points = []
            for x, y in zip(x_values, y_values):
                x = int(x)
                y = int(y)
                if 0 <= x < width and 0 <= y < height:
                    cv2.circle(image, (x, y), 10, point_color, -1)
                    line_points.append((x, y))
            points.extend(line_points)
        return points

    def pixel_to_global(self, y_pixel, x_pixel, cam_x, cam_y, cam_h, resolution, cam_angle):
        meter_to_global = 0.00001
        pixel_size_meters = math.tan(cam_angle) * cam_h / (resolution / 2)
        y_centered = resolution / 2 - y_pixel
        x_centered = x_pixel - resolution / 2
        y_meters = y_centered * pixel_size_meters
        x_meters = x_centered * pixel_size_meters
        pixel_global_x = cam_x + y_meters * meter_to_global
        pixel_global_y = cam_y + x_meters * meter_to_global
        return pixel_global_x, pixel_global_y

    def process_image(self, image_path, cam_x, cam_y, cam_h, resolution, cam_angle):
        img = cv2.imread(image_path)
        img_copy = img.copy()
        if img is None:
            print(f"Error: Unable to read image at {image_path}")
            return None, None, None, None, None

        preprocessed_image = self.preprocess_image(img)
        segmented_image = self.apply_kmeans(preprocessed_image)
        kernel = np.ones(DILATION_KERNEL_SIZE, np.uint8)
        segmented_image = cv2.dilate(segmented_image, kernel, iterations=DILATION_ITERATIONS)
        crop_rows = self.find_crop_rows(segmented_image)
        merged_lines = self.merge_lines(crop_rows, eps=0.5, min_samples=2)
        middle_lines = self.draw_middle_lines(img, merged_lines) if merged_lines else []
        points = self.draw_points_on_lines(img, middle_lines) if middle_lines else []
        global_points = [self.pixel_to_global(y, x, cam_x, cam_y, cam_h, resolution, cam_angle) for x, y in points]

        # Save processed images
        cv2.imwrite("processed_image_original.png", img_copy)
        cv2.imwrite("processed_image_preprocessed.png", preprocessed_image)
        cv2.imwrite("processed_image_segmented.png", segmented_image)

        if merged_lines:
            img_with_rows = img.copy()
            self.draw_lines(img_with_rows, merged_lines, (0, 0, 255))
            cv2.imwrite("processed_image_with_rows.png", img_with_rows)

            if middle_lines:
                img_with_middle_lines = img.copy()
                self.draw_lines(img_with_middle_lines, middle_lines, (255, 0, 0))
                cv2.imwrite("processed_image_with_middle_lines.png", img_with_middle_lines)

        return img, preprocessed_image, segmented_image, merged_lines, global_points


class CropRowDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Crop Row Detection App")
        self.create_widgets()
        self.detector = CropRowDetector()
        self.cap = None
        self.frame = None
        self.stop_event = threading.Event()
        self.output_dir = "C:/Users/lenovo/Documents/Mechatronics/Source_Directory"
        self.flask_thread = threading.Thread(target=self.run_flask_server, daemon=True)
        self.flask_thread.start()
        
    def run_flask_server(self):
        app = Flask(__name__)
    
        @app.route('/data', methods=['GET'])
        def get_data():
            # Return the latest global points data
            return jsonify({"points": self.detector.global_points})
    
        app.run(host='0.0.0.0', port=5000)
        
    def create_widgets(self):
        self.label = tk.Label(self.root, text="Crop Row Detection App", font=("Helvetica", 16))
        self.label.pack(pady=10)
    
        self.choose_image_button = tk.Button(self.root, text="Choose Image", command=self.choose_image)
        self.choose_image_button.pack(pady=5)
    
        self.start_live_button = tk.Button(self.root, text="Start Live Stream", command=self.start_live_stream)
        self.start_live_button.pack(pady=5)
    
        self.capture_button = tk.Button(self.root, text="Capture Photo", command=self.capture_photo)
        self.capture_button.pack(pady=5)
    
        self.param_frame = tk.Frame(self.root)
        self.param_frame.pack(pady=10)
    
        tk.Label(self.param_frame, text="Camera X:").grid(row=0, column=0, padx=5, pady=5)
        self.cam_x_entry = tk.Entry(self.param_frame)
        self.cam_x_entry.grid(row=0, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Y:").grid(row=1, column=0, padx=5, pady=5)
        self.cam_y_entry = tk.Entry(self.param_frame)
        self.cam_y_entry.grid(row=1, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Height:").grid(row=2, column=0, padx=5, pady=5)
        self.cam_h_entry = tk.Entry(self.param_frame)
        self.cam_h_entry.grid(row=2, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Resolution:").grid(row=3, column=0, padx=5, pady=5)
        self.resolution_entry = tk.Entry(self.param_frame)
        self.resolution_entry.grid(row=3, column=1, padx=5, pady=5)
    
        tk.Label(self.param_frame, text="Camera Angle:").grid(row=4, column=0, padx=5, pady=5)
        self.cam_angle_entry = tk.Entry(self.param_frame)
        self.cam_angle_entry.grid(row=4, column=1, padx=5, pady=5)
    
        self.process_image_button = tk.Button(self.root, text="Process Image", command=self.process_image)
        self.process_image_button.pack(pady=5)
    
        self.image_label = tk.Label(self.root)
        self.image_label.pack(pady=10)
        
    def choose_image(self):
        self.detector.image_path = filedialog.askopenfilename(initialdir=self.output_dir, title="Select an Image", filetypes=[("Image files", "*.jpg; *.jpeg; *.png")])
        self.display_image(self.detector.image_path)

    def start_live_stream(self):
        # Find the OBS Virtual Camera index
        self.camera_index = self.find_virtual_camera()
        if self.camera_index == -1:
            messagebox.showerror("Error", "OBS Virtual Camera not found.")
            return
        
        # Open the camera
        self.cap = cv2.VideoCapture(self.camera_index)
        if not self.cap.isOpened():
            messagebox.showerror("Error", f"Unable to open camera at index {self.camera_index}.")
            return
        
        self.stop_event.clear()
        self.show_live_stream()

    def show_live_stream(self):
        if self.stop_event.is_set():
            return
        ret, frame = self.cap.read()
        if ret:
            self.frame = frame
            cv2.imshow("Live Stream", frame)
        self.root.after(10, self.show_live_stream)

    def find_virtual_camera(self, start_index=1, max_index=10):
        """Try to find the OBS Virtual Camera by iterating over possible device indices."""
        for index in range(start_index, max_index):
            cap = cv2.VideoCapture(index)
            if cap.isOpened():
                print(f"Found camera at index {index}")
                
                # Check if this is the OBS Virtual Camera by capturing a frame and analyzing its content
                ret, frame = cap.read()
                if ret:
                    # Check if frame is not empty or perform other checks
                    if cv2.countNonZero(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)) > 0:
                        print(f"Camera at index {index} seems to be working.")
                        cap.release()
                        return index
                    else:
                        print(f"Camera at index {index} is not the OBS Virtual Camera.")
                else:
                    print(f"Unable to read frame from camera at index {index}.")
                
                cap.release()
    
        print("No OBS Virtual Camera found.")
        return -1

    def capture_photo(self):
        if self.frame is not None:
            image_path = os.path.join(self.output_dir, "captured_image.png")
            cv2.imwrite(image_path, self.frame)
            self.detector.image_path = image_path
            self.display_image(image_path)
            messagebox.showinfo("Image Captured", f"Image saved to {image_path}")

    def display_image(self, image_path):
        img = Image.open(image_path)
        img.thumbnail((400, 400))
        img = ImageTk.PhotoImage(img)
        self.image_label.configure(image=img)
        self.image_label.image = img

    def process_image(self):
        if self.detector.image_path is None:
            messagebox.showerror("Error", "Please select or capture an image first.")
            return
    
        # Retrieve parameters from entry fields
        try:
            cam_x = float(self.cam_x_entry.get())
            cam_y = float(self.cam_y_entry.get())
            cam_h = float(self.cam_h_entry.get())
            resolution = float(self.resolution_entry.get())
            cam_angle = float(self.cam_angle_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical values for all parameters.")
            return
    
        # Call function to process image
        img, preprocessed_image, segmented_image, merged_lines, global_points = self.detector.process_image(
            self.detector.image_path, cam_x, cam_y, cam_h, resolution, cam_angle
        )
    
        # Update global points
        self.detector.global_points = global_points
    
        if global_points:
            # Save map and plots
            map_filename = os.path.abspath("crop_row_map.html")
            self.create_map(map_filename, global_points)
            
            plot_filename = self.save_plots(self.detector.image_path)
            self.generate_html_report(plot_filename, map_filename)
        else:
            messagebox.showwarning("Warning", "No global points detected.")
        
    def create_map(self, map_filename, points):
        m = folium.Map(location=[points[0][0], points[0][1]], zoom_start=18)
    
        for point in points:
            folium.Marker(location=[point[0], point[1]]).add_to(m)
    
        m.save(map_filename)
    
    def on_closing(self):
        if self.cap is not None:
            self.stop_event.set()
            self.cap.release()
            cv2.destroyAllWindows()
        self.root.destroy()

    
    def save_plots(self, image_path):
        image, preprocessed_image, segmented_image, merged_lines, _ = self.detector.process_image(
            image_path,
            float(self.cam_x_entry.get()),
            float(self.cam_y_entry.get()),
            float(self.cam_h_entry.get()),
            float(self.resolution_entry.get()),
            float(self.cam_angle_entry.get())
        )
        
        if image is not None:
            # Save the plots
            fig, axes = plt.subplots(1, 5, figsize=(25, 8))
            axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[0].set_title('Original Image')
            axes[0].axis('off')

            axes[1].imshow(preprocessed_image, cmap='gray')
            axes[1].set_title('Preprocessed Image')
            axes[1].axis('off')

            axes[2].imshow(segmented_image, cmap='gray')
            axes[2].set_title('Segmented Image (K-Means)')
            axes[2].axis('off')

            if merged_lines:
                for slope, intercept in merged_lines:
                    x1 = 0
                    y1 = int(intercept)
                    x2 = image.shape[1]
                    y2 = int(slope * x2 + intercept)
                    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 10)

            axes[3].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[3].set_title('Detected Crop Rows')
            axes[3].axis('off')

            middle_lines = self.detector.draw_middle_lines(image, merged_lines) if merged_lines else []
            if middle_lines:
                for slope, intercept in middle_lines:
                    x1 = 0
                    y1 = int(intercept)
                    x2 = image.shape[1]
                    y2 = int(slope * x2 + intercept)
                    cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

            axes[4].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axes[4].set_title('Middle Lines')
            axes[4].axis('off')

            plt.tight_layout()
            plot_filename = os.path.join(self.output_dir, "plots.png")
            plt.savefig(plot_filename)
            plt.close()
            
            return plot_filename

    def generate_html_report(self, plot_filename, map_filename):
        html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Crop Row Detection Report</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    margin: 0;
                    padding: 0;
                    background-color: #f4f4f4;
                }}
                .container {{
                    width: 80%;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: #fff;
                    border-radius: 8px;
                    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                }}
                h1 {{
                    color: #4CAF50;
                    text-align: center;
                }}
                h2 {{
                    color: #333;
                }}
                .plot-container, .map-container {{
                    width: 100%;
                    max-width: 800px;
                    margin: 20px auto;
                    text-align: center;
                }}
                .plot-container img {{
                    max-width: 100%;
                    height: auto;
                    border-radius: 8px;
                }}
                .map-container iframe {{
                    width: 100%;
                    height: 500px;
                    border: none;
                    border-radius: 8px;
                }}
                .footer {{
                    text-align: center;
                    padding: 10px;
                    color: #666;
                }}
                .footer p {{
                    margin: 0;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Crop Row Detection Report</h1>
                <h2>Map Visualization</h2>
                <div class="map-container">
                    <iframe src="{map_filename}" title="Crop Row Map"></iframe>
                </div>
                <h2>Plots</h2>
                <div class="plot-container">
                    <img src="{plot_filename}" alt="Plots">
                </div>
                <div class="footer">
                    <p>Generated with love for agriculture.</p>
                </div>
            </div>
        </body>
        </html>
        """
        
        report_filename = os.path.join(self.output_dir, "report.html")
        with open(report_filename, "w") as file:
            file.write(html_content)
        
        webbrowser.open(report_filename)
        
            
    def process_image(self):
        if self.detector.image_path is None:
            messagebox.showerror("Error", "Please select or capture an image first.")
            return
    
        # Retrieve parameters from entry fields
        try:
            cam_x = float(self.cam_x_entry.get())
            cam_y = float(self.cam_y_entry.get())
            cam_h = float(self.cam_h_entry.get())
            resolution = float(self.resolution_entry.get())
            cam_angle = float(self.cam_angle_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numerical values for all parameters.")
            return
    
        # Call function to process image
        img, preprocessed_image, segmented_image, merged_lines, global_points = self.detector.process_image(
            self.detector.image_path, cam_x, cam_y, cam_h, resolution, cam_angle
        )
    
        if global_points:
            # Save map and plots
            map_filename = os.path.abspath("crop_row_map.html")
            self.create_map(map_filename, global_points)
            
            plot_filename = self.save_plots(self.detector.image_path)
            self.generate_html_report(plot_filename, map_filename)
        else:
            messagebox.showwarning("Warning", "No global points detected.")

                    
if __name__ == "__main__":
    root = tk.Tk()
    app = CropRowDetectionApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_closing)
    root.mainloop()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.9:5000
Press CTRL+C to quit
